In [ ]:
# =============================================================================
# Sección 1: Importación de Librerías y Configuración Inicial
# =============================================================================

# -----------------------------
# 1. Manipulación de datos
# -----------------------------
import pandas as pd
import numpy as np

# -----------------------------
# 2. Visualización
# -----------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# -----------------------------
# 3. Preprocesamiento y métricas
# -----------------------------
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error
)

# -----------------------------
# 4. Deep Learning y Series de Tiempo
# -----------------------------
import torch
from darts import TimeSeries
from darts.models import RNNModel
# RNNModel soporta model='RNN' | 'LSTM' | 'GRU'
from darts.dataprocessing.transformers import Scaler as DartsScaler

# -----------------------------
# 5. Optimización de Hiperparámetros
# -----------------------------
import optuna

# -----------------------------
# 6. Reproducibilidad
# -----------------------------
import random
import os

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -----------------------------
# 7. Configuración de estilo gráfico
# -----------------------------
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# -----------------------------
# 8. Supresión de warnings
# -----------------------------
import warnings
warnings.filterwarnings('ignore')

print(f"✅ Entorno configurado. Semilla global: {SEED}")


In [ ]:
# =============================================================================
# Sección 2: Carga y Exploración Inicial de Datos
# =============================================================================

# -----------------------------
# 1. Carga de archivos
# -----------------------------
from IPython.display import display

# Rutas relativas a tu proyecto
train_path      = '../../Datos/df_train.parquet'
test_path       = '../../Datos/df_test.parquet'
submit_path     = '../../Datos/sample_submission.csv'

# Lectura de los datos
train_df        = pd.read_parquet(train_path)     # Datos de entrenamiento (2015–2021)
test_df         = pd.read_parquet(test_path)      # Datos de test (2022, sin 'dengue')
submission_df   = pd.read_csv(submit_path)        # Formato de envío



# ----------------------------------------------------------
# Conversión forzada de columnas clave a tipo numérico
# ----------------------------------------------------------
# Si hay valores no convertibles, pasarán a NaN para que no rompan los agregados
train_df['dengue'] = pd.to_numeric(train_df['dengue'], errors='coerce')
train_df['semana'] = pd.to_numeric(train_df['semana'], errors='coerce')

# Verificamos que ahora sean numéricos
print(train_df[['dengue','semana']].dtypes)
# Y cuántos NaN se generaron (opcional)
print("Nulos en 'dengue':", train_df['dengue'].isna().sum())
print("Nulos en 'semana':", train_df['semana'].isna().sum())



# -----------------------------
# 2. Inspección básica
# -----------------------------
print("➡️  Primeras filas de train_df:")
display(train_df.head())

print("\n➡️  Estructura y tipos de datos de train_df:")
train_df.info()

print("\n➡️  Primeras filas de test_df:")
display(test_df.head())

print("\n➡️  Estructura y tipos de datos de test_df:")
test_df.info()

print("\n➡️  Muestra de sample_submission.csv:")
display(submission_df.head())

# -----------------------------
# 3. Valores faltantes
# -----------------------------
print("\n➡️  Valores faltantes en train_df:")
missing_train = train_df.isnull().sum()
print(missing_train[missing_train > 0])

print("\n➡️  Valores faltantes en test_df:")
missing_test = test_df.isnull().sum()
print(missing_test[missing_test > 0])

# Visualización rápida de missing (solo variables con >0 faltantes)
if (missing_train > 0).any():
    fig, ax = plt.subplots()
    missing_train[missing_train > 0].plot.bar(ax=ax)
    ax.set_title("Valores faltantes en train_df")
    ax.set_ylabel("Cantidad")
    plt.xticks(rotation=45)
    plt.show()

# -----------------------------
# 4. Estadísticas descriptivas
# -----------------------------
print("\n➡️  Estadísticas descriptivas de train_df:")
display(train_df.describe())

# Estadísticas categóricas (por ejemplo, conteo de barrios)
print("\n➡️  Número de registros por barrio (id_bar):")
display(train_df['id_bar'].value_counts().sort_index())

# Distribución de la variable objetivo 'dengue'
fig, ax = plt.subplots()
sns.histplot(train_df['dengue'], bins=30, kde=True, ax=ax)
ax.set_title("Distribución de casos de dengue (train_df)")
ax.set_xlabel("Casos de dengue por semana")
plt.show()


In [ ]:
# =============================================================================
# Sección 3: Análisis Exploratorio Enfocado en Series Temporales
# =============================================================================

# -----------------------------
# 3.1 Análisis por Barrio
# -----------------------------
# 3.1.1 – Crear columna 'date' a partir de año y semana (lunes de la semana)
train_df['date'] = train_df.apply(
    lambda r: pd.to_datetime(f"{r.anio}-{int(r.semana):02d}-1", format="%Y-%W-%w"),
    axis=1
)

# 3.1.2 – Pivot: filas=fecha, columnas=barrio, valores=casos dengue
ts_barrio = train_df.pivot_table(
    index='date',
    columns='id_bar',
    values='dengue',
    aggfunc='sum'
).sort_index()

# 3.1.3 – Identificar barrios con mayor y menor incidencia total
total_por_barrio = train_df.groupby('id_bar')['dengue'].sum().sort_values()
top5_barrio    = total_por_barrio.tail(5).index.tolist()
bottom5_barrio = total_por_barrio.head(5).index.tolist()

print("Top 5 barrios con más casos acumulados:", top5_barrio)
print("Bottom 5 barrios con menos casos acumulados:", bottom5_barrio)

# 3.1.4 – Visualizar series temporales para top5 y bottom5
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
for b in top5_barrio:
    axes[0].plot(ts_barrio.index, ts_barrio[b], label=f"Barrio {b}")
axes[0].set_title("Evolución semanal de dengue — Top 5 barrios")
axes[0].legend(loc='upper left')

for b in bottom5_barrio:
    axes[1].plot(ts_barrio.index, ts_barrio[b], label=f"Barrio {b}")
axes[1].set_title("Evolución semanal de dengue — Bottom 5 barrios")
axes[1].legend(loc='upper left')

plt.xlabel("Fecha")
plt.tight_layout()
plt.show()

# 3.1.5 – Incidencia total por barrio (barras)
fig, ax = plt.subplots(figsize=(12, 6))
total_por_barrio.plot.bar(ax=ax)
ax.set_title("Casos acumulados de dengue por barrio (2015–2021)")
ax.set_xlabel("ID de Barrio")
ax.set_ylabel("Casos totales")
plt.xticks(rotation=90)
plt.show()


# -----------------------------
# 3.2 Análisis de Estacionalidad y Tendencias
# -----------------------------
from statsmodels.tsa.seasonal import seasonal_decompose

# Tomamos el barrio con más casos para ejemplo de descomposición
b0 = top5_barrio[-1]
serie_b0 = ts_barrio[b0].asfreq('W-MON').fillna(0)

# Descomposición STL (tendencia, estacionalidad, residuo)
decomp = seasonal_decompose(serie_b0, model='additive', period=52)

fig = decomp.plot()
fig.set_size_inches(14, 8)
plt.suptitle(f"Descomposición de la serie de dengue — Barrio {b0}", y=1.02)
plt.show()

# Estacionalidad promedio por semana del año
seasonal_week = train_df.groupby('semana')['dengue'].mean()
fig, ax = plt.subplots(figsize=(12, 4))
sns.lineplot(x=seasonal_week.index, y=seasonal_week.values, ax=ax)
ax.set_title("Promedio semanal de casos de dengue (2015–2021)")
ax.set_xlabel("Semana del año")
ax.set_ylabel("Casos promedio")
plt.show()


# -----------------------------
# 3.3 Análisis de Correlaciones Temporales
# -----------------------------
from statsmodels.graphics.tsaplots import plot_acf

# 3.3.1 – Autocorrelación de la serie agregada de dengue
agg_series = train_df.set_index('date')['dengue'].resample('W-MON').sum()
fig, ax = plt.subplots(figsize=(10, 4))
plot_acf(agg_series.fillna(0), lags=24, ax=ax)
ax.set_title("Autocorrelación de casos semanales de dengue (agregado)")
plt.show()

# -----------------------------
# 3.3.2 – Correlación cruzada simple entre lluvia media y dengue
# -----------------------------
# 1. Seleccionamos sólo las columnas numéricas de interés
weekly_df = (
    train_df
    .set_index('date')[['dengue', 'lluvia_mean']]   # sólo numéricas
    .resample('W-MON')                              # muestreo semanal (lunes)
    .mean()                                         # media de cada columna
    .fillna(0)                                      # rellenar NaN con 0
)

# 2. Calculamos las correlaciones con distintos lags
ccf_vals = [weekly_df['lluvia_mean'].shift(l).corr(weekly_df['dengue']) for l in range(9)]

# 3. Visualizamos
fig, ax = plt.subplots(figsize=(8, 4))
sns.barplot(x=list(range(9)), y=ccf_vals, ax=ax)
ax.set_title("Correlación (lag) entre lluvia_mean y dengue")
ax.set_xlabel("Desfase de semanas")
ax.set_ylabel("Coeficiente de correlación")
plt.show()



# -----------------------------
# 3.4 Determinación de Ventanas Temporales
# -----------------------------
# Usamos la autocorrelación para ver hasta qué lag es significativo
acf_vals = agg_series.fillna(0).autocorr(lag=1)
# Comparamos varios lags
lags = list(range(1, 21))
acf_lags = [agg_series.autocorr(lag=l) for l in lags]

fig, ax = plt.subplots(figsize=(10, 4))
sns.lineplot(x=lags, y=acf_lags, marker='o', ax=ax)
ax.axhline(0.2, color='red', linestyle='--', label='Umbral 0.2')
ax.set_title("ACF: elección de ventana temporal óptima (semanas)")
ax.set_xlabel("Lag (semanas)")
ax.set_ylabel("Autocorrelación")
ax.legend()
plt.show()

print("➡️ Observación: la autocorrelación cae por debajo de 0.2 alrededor de las 8–10 semanas,")
print("   lo que sugiere considerar ventanas de entrada de longitud ~8–10 semanas.")


In [ ]:
# =============================================================================
# Sección 4: Análisis y Selección de Variables
# =============================================================================

# -----------------------------
# 4.1 Importancia de Variables
# -----------------------------

# 4.1.1 – Identificar columnas numéricas predictoras (excluimos id, date y target)
exclude_cols = ['id', 'id_bar', 'anio', 'date', 'dengue']
numeric_cols = [
    c for c in train_df.columns
    if pd.api.types.is_numeric_dtype(train_df[c]) and c not in exclude_cols
]

# 4.1.2 – Correlación de cada variable numérica con la variable objetivo 'dengue'
corr_with_target = train_df[numeric_cols + ['dengue']].corr()['dengue'].drop('dengue')
corr_with_target = corr_with_target.sort_values()

# 4.1.3 – Visualizar correlaciones
fig, ax = plt.subplots(figsize=(10, 6))
corr_with_target.plot.barh(ax=ax)
ax.set_title("Correlación de variables numéricas con dengue")
ax.set_xlabel("Coeficiente de correlación")
plt.tight_layout()
plt.show()

# 4.1.4 – Importancia usando Random Forest
from sklearn.ensemble import RandomForestRegressor

# Definimos X e y (eliminamos filas con NaN en predictors o target)
rf_df = train_df.dropna(subset=numeric_cols + ['dengue'])
X = rf_df[numeric_cols]
y = rf_df['dengue']

# Instanciamos y ajustamos el modelo
rf = RandomForestRegressor(
    n_estimators=100,
    random_state=SEED,
    n_jobs=-1
)
rf.fit(X, y)

# Extraemos importancias
importances = pd.Series(rf.feature_importances_, index=numeric_cols)
importances = importances.sort_values()

# 4.1.5 – Visualizar importancias de Random Forest
fig, ax = plt.subplots(figsize=(10, 6))
importances.plot.barh(ax=ax)
ax.set_title("Importancia relativa de variables (Random Forest)")
ax.set_xlabel("Importancia")
plt.tight_layout()
plt.show()


# -----------------------------
# 4.2 Reducción de Dimensionalidad (PCA)
# -----------------------------
from sklearn.decomposition import PCA

# 4.2.1 – Escalar variables antes de PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4.2.2 – Ajustar PCA y obtener varianza explicada
pca = PCA()
pca.fit(X_scaled)

explained_var = pca.explained_variance_ratio_
cum_var = np.cumsum(explained_var)

# 4.2.3 – Visualizar varianza explicada acumulada
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(range(1, len(cum_var) + 1), cum_var, marker='o')
ax.axhline(0.90, linestyle='--', label='90% Varianza')
ax.set_title("Varianza explicada acumulada por componentes PCA")
ax.set_xlabel("Número de componentes")
ax.set_ylabel("Varianza explicada acumulada")
ax.legend()
plt.tight_layout()
plt.show()

# 4.2.4 – Si es necesario, seleccionar componentes o eliminar variables redundantes
# Identificación de pares de variables con alta correlación (> 0.9)
corr_matrix = pd.DataFrame(X_scaled, columns=numeric_cols).corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.90)]
print(f"Variables con correlación > 0.90 (a considerar para eliminación):\n{to_drop}")

# Crear nuevo conjunto de características sin las redundantes
X_reduced = X.drop(columns=to_drop)

print(f"Tamaño original de X: {X.shape[1]} variables")
print(f"Tamaño reducido de X: {X_reduced.shape[1]} variables")


In [ ]:
# =============================================================================
# Sección 5: Preprocesamiento para Series Temporales
# =============================================================================

# -----------------------------
# 5.1: Escalado de Variables
# -----------------------------
# Recalculamos la lista de predictoras numéricas reducidas (de la Sección 4)
numeric_cols_reduced = [c for c in numeric_cols if c not in to_drop]

# Usamos MinMaxScaler para llevar todas las variables al rango [0,1]
scaler = MinMaxScaler()
# Ajustamos y transformamos sólo las predictoras numéricas
train_scaled = train_df.copy()
train_scaled[numeric_cols_reduced] = scaler.fit_transform(
    train_df[numeric_cols_reduced]
)

# -----------------------------
# 5.2: Creación de Características Temporales
# -----------------------------
# Definimos lags de dengue y rolling windows
lags = [1, 2, 4, 8]
rolling_windows = [4, 8]

for lag in lags:
    col = f"dengue_lag_{lag}"
    train_scaled[col] = (train_scaled
                         .groupby('id_bar')['dengue']
                         .shift(lag))

for window in rolling_windows:
    col = f"dengue_rollmean_{window}"
    train_scaled[col] = (train_scaled
                         .groupby('id_bar')['dengue']
                         .rolling(window=window, min_periods=1)
                         .mean()
                         .reset_index(level=0, drop=True))

# -----------------------------
# 5.3: Limpieza de NaNs
# -----------------------------
# Eliminamos filas donde alguna feature temporal o target sea NaN
feature_cols = numeric_cols_reduced + \
               [f"dengue_lag_{lag}" for lag in lags] + \
               [f"dengue_rollmean_{w}" for w in rolling_windows]
train_scaled = train_scaled.dropna(subset=feature_cols + ['dengue'])

# -----------------------------
# 5.4: Preparación de TimeSeries de Darts (índices únicos)
# -----------------------------
series_dict = {}

for barrio in train_scaled['id_bar'].unique():
    # 1) Filtrar barrio y agrupar por fecha para tener un índice único
    df_bar = (
        train_scaled[train_scaled['id_bar'] == barrio]
        .groupby('date')[['dengue'] + feature_cols]
        .mean()               # puedes cambiar a .sum() si prefieres sumar los casos
        .reset_index()
    )

    # 2) Construir la serie objetivo
    ts_target = TimeSeries.from_dataframe(
        df_bar,
        time_col='date',
        value_cols='dengue',
        freq='W-MON'         # asegura frecuencia semanal sin duplicados
    )
    # 3) Construir covariables
    ts_cov = TimeSeries.from_dataframe(
        df_bar,
        time_col='date',
        value_cols=feature_cols,
        freq='W-MON'
    )

    # 4) Guardar en el diccionario
    series_dict[barrio] = {
        'target': ts_target,
        'covariates': ts_cov
    }

# -----------------------------
# 5.5: División Temporal Train/Validation
# -----------------------------
train_series = {}
val_series   = {}
split_date   = pd.Timestamp('2020-12-31')

for barrio, seqs in series_dict.items():
    tgt, cov = seqs['target'], seqs['covariates']
    train_tgt, val_tgt = tgt.split_after(split_date)
    train_cov, val_cov = cov.split_after(split_date)
    train_series[barrio] = {'target': train_tgt, 'covariates': train_cov}
    val_series[barrio]   = {'target': val_tgt,   'covariates': val_cov}

print(f"✅ Series Darts creadas para {len(series_dict)} barrios con índices únicos.")
print(f"   – Training hasta {split_date.date()}, validación desde {split_date.date() + pd.Timedelta(days=1)}.")


# -----------------------------
# 5.5: División Temporal Train/Validation
# -----------------------------
train_series = {}
val_series   = {}
# Usamos todo 2015–2020 para entrenar y 2021 como validación
split_date = pd.Timestamp('2020-12-31')

for barrio, seqs in series_dict.items():
    tgt, cov = seqs['target'], seqs['covariates']
    train_tgt, val_tgt = tgt.split_after(split_date)
    train_cov, val_cov = cov.split_after(split_date)
    train_series[barrio] = {'target': train_tgt, 'covariates': train_cov}
    val_series[barrio]   = {'target': val_tgt,   'covariates': val_cov}

print(f"✅ Preprocesamiento completo. Series Darts creadas para {len(series_dict)} barrios.")
print(f"   – Training hasta {split_date.date()}, validación desde {split_date.date() + pd.Timedelta(days=1)}.")


In [ ]:
# =============================================================================
# Sección 6: Implementación de Modelos con DARTS
# =============================================================================

# -----------------------------
# 6.1 Parámetros comunes
# -----------------------------
input_chunk_length  = 8    # semanas de historial
output_chunk_length = 1    # horizonte de predicción en semanas

common_kwargs = dict(
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
    batch_size=32,
    n_epochs=50,
    random_state=SEED,
    save_checkpoints=False,
    pl_trainer_kwargs={"accelerator": "gpu" if torch.cuda.is_available() else "cpu"}
)

# -----------------------------
# 6.2 Función helper para entrenamiento y predicción
# -----------------------------
def train_and_predict(model, train_s, covariates, val_s):
    """
    Ajusta `model` sobre train_s usando covariates como future_covariates,
    luego predice sobre val_s durante n_timesteps.
    """
    model.fit(
        series=train_s['target'],
        future_covariates=covariates
    )
    n_steps = val_s['target'].n_timesteps
    return model.predict(
        n=n_steps,
        future_covariates=covariates
    )

# -----------------------------
# 6.3 Selección de un barrio de ejemplo
# -----------------------------
b0       = list(train_series.keys())[0]
train_s  = train_series[b0]
val_s    = val_series[b0]
full_cov = series_dict[b0]['covariates']  # debe arrancar al menos input_chunk_length semanas antes

# -----------------------------
# 6.4 RNN básica
# -----------------------------
rnn_model = RNNModel(
    model="RNN",
    hidden_dim=64,      # parámetro correcto para tu versión de Darts
    n_rnn_layers=2,
    dropout=0.1,
    model_name="RNN_basic",
    **common_kwargs
)
rnn_pred = train_and_predict(rnn_model, train_s, full_cov, val_s)

# -----------------------------
# 6.5 LSTM
# -----------------------------
lstm_model = RNNModel(
    model="LSTM",
    hidden_dim=64,
    n_rnn_layers=2,
    dropout=0.1,
    model_name="LSTM_basic",
    **common_kwargs
)
lstm_pred = train_and_predict(lstm_model, train_s, full_cov, val_s)

# -----------------------------
# 6.6 GRU
# -----------------------------
gru_model = RNNModel(
    model="GRU",
    hidden_dim=64,
    n_rnn_layers=2,
    dropout=0.1,
    model_name="GRU_basic",
    **common_kwargs
)
gru_pred = train_and_predict(gru_model, train_s, full_cov, val_s)

# -----------------------------
# 6.7 Visualización comparativa
# -----------------------------
import matplotlib.dates as mdates

plt.figure(figsize=(12, 6))
train_s['target'].plot(label="Histórico")
val_s['target'].plot(label="Validación (real)", lw=2)
rnn_pred.plot(label="Predicción RNN")
lstm_pred.plot(label="Predicción LSTM")
gru_pred.plot(label="Predicción GRU")

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))

plt.title(f"Comparación de Modelos – Barrio {b0}")
plt.xlabel("Fecha")
plt.ylabel("Casos semanales de dengue")
plt.legend()
plt.tight_layout()
plt.show()


In [11]:
# =============================================================================
# Sección 7: Optimización de Hiperparámetros con Optuna
#           — filtrado de NaNs antes de calcular MSE
# =============================================================================

import optuna
import torch
import numpy as np
from sklearn.metrics import mean_squared_error
from darts.models import RNNModel

# ------------------------------------------------------------------------
# Datos de ejemplo (un barrio)
# ------------------------------------------------------------------------
b0       = list(train_series.keys())[0]
train_s  = train_series[b0]
val_s    = val_series[b0]
full_cov = series_dict[b0]['covariates']

# ------------------------------------------------------------------------
# Construcción de la función objetivo
# ------------------------------------------------------------------------
def make_objective(model_type: str):
    def objective(trial):
        # 1) Espacio de búsqueda
        hidden_dim = trial.suggest_int("hidden_dim", 16, 128)
        n_layers   = trial.suggest_int("n_rnn_layers", 1, 3)
        dropout    = trial.suggest_float("dropout", 0.0, 0.5)
        lr         = trial.suggest_loguniform("lr", 1e-4, 1e-2)
        batch_sz   = trial.suggest_categorical("batch_size", [16, 32, 64])

        # 2) Instanciar el modelo
        model = RNNModel(
            model=model_type,
            hidden_dim=hidden_dim,
            n_rnn_layers=n_layers,
            dropout=dropout,
            optimizer_cls=torch.optim.Adam,
            optimizer_kwargs={"lr": lr},
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            batch_size=batch_sz,
            n_epochs=20,
            random_state=SEED,
            pl_trainer_kwargs={"accelerator": "gpu" if torch.cuda.is_available() else "cpu"},
            save_checkpoints=False
        )

        # 3) Entrenar
        model.fit(
            series=train_s["target"],
            future_covariates=full_cov
        )

        # 4) Predecir en validación
        n_steps = val_s["target"].n_timesteps
        pred_ts = model.predict(n=n_steps, future_covariates=full_cov)

        # 5) Extraer y alinear arrays
        y_true = np.array(val_s["target"].values()).reshape(-1)
        y_pred = np.array(pred_ts.values()).reshape(-1)

        # 6) Filtrar NaNs
        mask = np.isfinite(y_true) & np.isfinite(y_pred)
        if mask.sum() == 0:
            return float("inf")

        y_true, y_pred = y_true[mask], y_pred[mask]

        # 7) Calcular y devolver MSE
        return mean_squared_error(y_true, y_pred)
    return objective

# ------------------------------------------------------------------------
# Ejecutar estudios y capturar solo resultados finales
# ------------------------------------------------------------------------
results = {}
sampler = optuna.samplers.TPESampler(seed=SEED)
pruner  = optuna.pruners.MedianPruner()

for model_type in ["RNN", "LSTM", "GRU"]:
    study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
    try:
        study.optimize(make_objective(model_type), n_trials=20, show_progress_bar=False)
        results[model_type] = (study.best_params, study.best_value)
    except Exception as e:
        # Mostrar el error solo si ocurre
        print(f"❗ Error optimizando {model_type}: {e}")

# ------------------------------------------------------------------------
# Imprimir únicamente los mejores hiperparámetros y su MSE numérico
# ------------------------------------------------------------------------
for mt, (params, mse) in results.items():
    print(f"▶️ Mejor {mt}: {params} — MSE: {mse:.4f}")


[I 2025-06-15 11:12:05,672] A new study created in memory with name: no-name-88645b6e-ba4c-444a-b39f-5d02f442531d
ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 18.5 K | train
6 | V               | Linear           | 59     | train
-------------------------------------------------------------
18.6 K    Trainable params
0         Non-trainable params
18.6 

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2025-06-15 11:12:08,905] Trial 0 finished with value: inf and parameters: {'hidden_dim': 58, 'n_rnn_layers': 3, 'dropout': 0.36599697090570255, 'lr': 0.0015751320499779737, 'batch_size': 16}. Best is trial 0 with value: inf.
ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 41.4 K | train
6 | V               | Linear           | 114    | train
-----------

...

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2025-06-15 11:13:42,029] Trial 19 finished with value: inf and parameters: {'hidden_dim': 80, 'n_rnn_layers': 2, 'dropout': 0.3909729109925136, 'lr': 0.00026782584648770623, 'batch_size': 64}. Best is trial 0 with value: inf.


▶️ Mejor RNN: {'hidden_dim': 58, 'n_rnn_layers': 3, 'dropout': 0.36599697090570255, 'lr': 0.0015751320499779737, 'batch_size': 16} — MSE: inf
▶️ Mejor LSTM: {'hidden_dim': 103, 'n_rnn_layers': 1, 'dropout': 0.0027610585618011996, 'lr': 0.004274869455295219, 'batch_size': 64} — MSE: inf
▶️ Mejor GRU: {'hidden_dim': 124, 'n_rnn_layers': 1, 'dropout': 0.24862425294619273, 'lr': 0.0003997206819121809, 'batch_size': 64} — MSE: inf
